## Notes on ice thickness estimation from airborne LiDAR
Adam Steer, May 2016.
adam.d.steer@gmail.com

For this project, we have a bunch of LIDAR points georeferenced relative to an ITRF08 ellipsoid. What we need is the height of each point relative to sea level.

But where exactly is the sea surface? We can estimate it from a gravity model, plus a tide model, plus some knowledge of dynamic sea surface topography (wind pressure etc. ).

Or we can use a sensor-based approach, since we know some points are returns from water, or extremely thin new ice.

This document describes the second approach.

Using a small subset of LiDAR as an example, this notebook steps through the process of estimating sea ice thickness.

### First, import some libraries we will need

In [1]:
#setup
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

#Used for some functions involving finding points within
# N metres of any other point
from sklearn import neighbors as nb

#Testing a Kriging library from sklearn. Memory hungry!
#from sklearn.gaussian_process import GaussianProcess

from scipy.interpolate import SmoothBivariateSpline
#Scipy's least squares bivariate spline was tested
# but less functional than it's higher level SmoothBivariateSpline
#from scipy.interpolate import LSQBivariateSpline

#used to find the N closest points to neighbourhood median, 
# for filtering 'water' keypoints
from heapq import nsmallest

## Function declarations

In [4]:
# this function is the snow depth model. Parameters come from Steer et al (2016):
#Estimating snow depth from altimery for East Antarctic pack ice

def compute_zs(tf, s_i, tf_uncert):
    """
    Take in the total freeboard (tf, float array), slope and intercept for an empirical model
    of snow depth from elevation (s_i, tuple) and the total freeboard uncertainty
    (tf_uncert, float array)

    Return a snow depth, with an associated uncertainty.
    zs, uncert = compute_zs(tf, ([slope, intercept]), tf_uncert)
    """
    
    zs = (params[0] * tf) + s_i[1]

    zs_uncert = s_i[0] * tf_uncert

    return zs, zs_uncert


In [5]:
# next define the model for estimating ice thickness from total freeboard, snow depth
# and some density parameters:

def compute_zi(tf, zs, d_ice, d_water, d_snow, sd_tf, sd_zs, sd_dsnow, \
               sd_dice, sd_dwater):
    """"
    sea ice thickness from elevation, and propagation of uncertainties
    after Kwok, 2010; kwok and cunningham, 2008
    equations:
    4: sea ice thickness from elevation
    6: taylor series expansion of variance/covariance propagation using
    partial derivatives
    8, 9 and 10: partial derivatives used in the taylor series
    """
    zi = (d_water / (d_water-d_ice)) * tf - ((d_water-d_snow) / \
         (d_water - d_ice)) * zs

    zi_uncert = sd_tf**2 * (d_water / (d_water - d_ice))**2 + \
           sd_zs**2 * ((d_snow - d_water) / (d_water - d_ice))**2 + \
           sd_dsnow**2 * (zs / (d_water - d_ice))**2 + \
           sd_dice**2 * (tf /  (d_water - d_ice))**2 + \
           sd_dwater**2 * (((-d_ice * tf) + ((d_ice-d_snow) * zs)) / \
           (d_water - d_ice)**2)**2

    return zi, zi_uncert


In [6]:
# A function to build a KD-tree
def build_tree(pointcloud, leafsize):
    return nb.KDTree(pointcloud, leaf_size=leafsize)

In [8]:
#part one of fiding water - intensity percentile based low pass filter
def find_low_intens(intens,pntile):
    """
    Take in some return intensity data (array) and a percentile (float)
    Return the indices of chosen percentile of intensity values
    Some of these will be returns on ice, which need to be exlcuded.
    A number-of-neighbours test is used to do this.
    """
    li_idx =  np.where(intens <= np.percentile(intens, pntile))
    return list(li_idx[0])

In [7]:
# part two of finding sea level keypoints
def find_lowpoints(xyz, leafsize, nhood):
    """
    Pass in an array of elevation vlues and an integer  number of points to use
    as a neighbourhood.
    Get back the indices of 10 elevation values closest to the neighbourhooed median
    for neighbourhoods with more than nhood*4 points
    It's not really finding low points, so the function needs renaming.
    """    
    
    low_tree = build_tree(xyz, leafsize)
    nhoods = low_tree.query_radius(xyz, r=nhood)
    point_filt = []   
  
    for nh in nhoods:
        #print(nhood)
        #print(xyz[nhood,2])
        # we want indices in the input cloud
        # of the 10 values closest to the nhood median
        if len(nh) > 3*nhood:
            npoints = sorted(enumerate(xyz[nh[:],2]),\
                      key=lambda x:abs(np.median(xyz[nh[:],2])-x[1]))[:10]
            #print(npoints)
            for point in npoints:
                #print(point[0])
                point_filt.append(nh[point[0]])
                #print(xyz[point[0],:])

            #point_filt.append(nh[find_nearest(xyz[nh[:],2], np.median(xyz[nh[:],2]))])
        #print(new_z[i])
    #return a list of indices
    return point_filt


In [9]:
# gluing it together, using the two functions above
def find_water_keys(xyzi, e_p, i_p):
    """
    Pass in:
    - a set of X,Y,Z and Intensity points
    - the number of points to use as a neighbourhood for choosing low elevation
    - a percentile level for intensity thresholding
    Get back:
    - a set of XYZI points corresponding to 'local sea level'
    """
    #find lowest intensity values
    low_intens_inds = find_low_intens(xyzi[:, 3], i_p)
    li_xyzi = xyzi[low_intens_inds, :]

    #find low elevations
    low_elev_inds = find_lowpoints(li_xyzi[:, 2], e_p)
    low_xyzi = li_xyzi[low_elev_inds, :]
    #low_xyzi = np.squeeze(low_xyzi)

    return low_xyzi

In [ ]:
#now we have water keys, this function fits a 2D spline, the adjusts the points by evaluating the spline
# at keypoints and subtracting the result from LiDAR points. The derived splines are also evaluated at *every*
# lidar point, and the result subtracted - so there are some careful considerations about extrapolation from
# polynomial splines and the shape of data being fitted. In this function, X/Y coordinates are normalised such that
# [0 < X < 1] and [0 < Y < 1] before spline fitting. This reduces (but not destroys) the likelihood that
# extrapolation of values outside the keypoint range results in wildly spurious values. As a precondition,
# input data are clipped such that the along-track dimension is always within the range of keypoints
# (ie swath chunks start and end over water).

def f_points(in_, params, dims, smth):
    '''
    find open water points using find_water_keys
    fit a surface to central tendency of keypoints
    adjust the rest of a LiDAR swath by water keypoint values
    In this method, coordinates are normalised so that 
    interpolation occurs in an almost-gridded environment which data pretty
    much fill.
    
    Best applied with a lot of smoothing
    '''
    #normalise input data to a unit block
    #http://stackoverflow.com/questions/3864899/resampling-irregularly-spaced-data-to-a-regular-grid-in-python
    # first get min/max of points
    xmin = np.min(in_[:,0])
    ymin = np.min(in_[:,1])    
    
    #translate to starting at [0,0]
    t_x = in_[:,0]-xmin
    t_y = in_[:,1]-ymin

    #normalise coordinates in each direction    
    xmax = np.max(t_x)
    ymax = np.max(t_y)
    norm_x = t_x/xmax
    norm_y = t_y/ymax
    
    #set up a translated world-space array to send to water point finding
    to_waterkeys = np.column_stack([t_x, t_y, in_[:,2], in_[:,3]])
    #print(to_waterkeys)
    
    #find water points   
    xyz_fp = np.squeeze(find_water_keys(to_waterkeys, params[0], params[1]))
    
    #translate the results to the same normalised coordinate system
    norm_fit_x = xyz_fp[:,0]/xmax
    norm_fit_y = xyz_fp[:,1]/ymax
    
    #check extents
    print('min. norm fit Y: {}, max. norm fit Y: {}'.format(min(norm_fit_y), max(norm_fit_y)))    
    
    #another 2D spline, pretty much the same as SmoothBivariateSpline
    #xcoord = np.linspace(0, 1, 0.1)
    #ycoord = np.linspace(0, 1, 0.1)
    #this_f = LSQBivariateSpline(norm_fit_x, norm_fit_y,
    #                            xyz_fp[:, 2], xcoord, ycoord)
    
    #this one is the winner right now.
    #fit a spline using normalised XY coordinates
    this_f = SmoothBivariateSpline(norm_fit_x, norm_fit_y,
                                   xyz_fp[:, 2], kx=dims[0], ky=dims[1],
                                   bbox=[0,1,0,1], s = smth*len(norm_fit_x))
                                   
    #evaluate the function at real-space coordinates
    #fpoints_mod = this_f.ev(xyz_fp[:, 0], xyz_fp[:, 1])
    #or normalised? Normalised, of course!
    fpoints_mod = this_f.ev(norm_fit_x, norm_fit_y)
    
    #subtract evaluated spline heights from the LiDAR points
    adjusted_points =  xyz_fp[:, 2] - fpoints_mod
    
    #one more filter - kill extreme values (+- 3 * sd)
    e_f = np.where((adjusted_points >= 0-3*np.std(adjusted_points)) & (adjusted_points<= 0+3*np.std(adjusted_points)))

    print('mean fitpoint adjustment (Z): {}'.format(np.mean(fpoints_mod[e_f[0]])))
    
    #translate fit points back!
    fitpoints_mod = np.column_stack([xyz_fp[e_f[0], 0]+xmin, xyz_fp[e_f[0], 1]+ymin,
                                    adjusted_points[e_f[0]]])
    #evaluate the surface at 
    #z_mod = this_f.ev(in_[:, 0], in_[:, 1])
    
    #normalised coordinates
    z_mod = this_f.ev(norm_x, norm_y)
    
    coeffs = this_f.get_coeffs()
    resid = this_f.get_residual()    
    
    #remove predicted values from elevations
    xyzi_mod = np.column_stack([in_[:, 0], in_[:, 1],
                               in_[:, 2] - z_mod, in_[:, 3]])
                               
    return xyzi_mod, fitpoints_mod, resid, coeffs

In [11]:
# finally, a function to make n-radius smoothed points, using whatever
# function works best. Here, mean, median and std are used. Median values
# The first two can be used to smooth noisy points. The third is a proxy for
# roughness in the neighbourhood chosen.

def n_filter(pointcloud, tree, radius):
    '''
    takes in a point cloud (xyzi), a kDtree (generated in build_tree),
    and a neighbourhood.
    returns the standard deviation of points within (radius) metres
    of each point as a new point list.
    '''
    nhoods = tree.query_radius(pointcloud[:,0:3], r=radius)
    n_stats = []
    i = 0
    for nhood in nhoods:
        #print(nhood)
        n_stats.append([np.mean(pointcloud[nhood[:],2]),\
                      np.median(pointcloud[nhood[:],2]),\
                      np.std(pointcloud[nhood[:],2])])
        #print(pointcloud[i,:])
        #print(new_z[i])
        i += 1
    return n_stats

## Applying it all! Setting up reference points and levelling the data

### getting some data

In [1]:
#setting up a file path for IO
infile_dir = '/media/adam/data/is6_f11/lidar.matlab/swaths/'
infile = 'is6_f11_pass1_aa_nr2_522816_523019_c.xyz'
outfilename = 'is6_f11_pass1_aa_nr2_522816_523019_zi_data.xyz'

In [ ]:
input_points = input_points[input_points[:,3] < -5,:]

# crop any that are likely to be air returns
input_points = input_points[input_points[:,3] < -5,:]

input_points = input_points[1:200000,:]

#grab an xyzi subset to work with
xyzi_ = input_points[:, 1:5]

print('x limits: {} - {}'.format(np.min(xyzi_[:,0]), np.max(xyzi_[:,0])))
print('y limits: {} - {}'.format(np.min(xyzi_[:,1]), np.max(xyzi_[:,1])))
print('total points: {}'.format(len(xyzi_[:,1])))

...now we have a set of LiDAR points in local coordinates - oriented such that the X-Y plane is aligned with 'across track', and 'along track' respectively. Essentially pretending the aircraft flew approximately along the Y axis - for interpolation/extrapolation reasons.

### next, set up some snow and ice model parameters

In [2]:
#density parameters, should add as a data dictionary
d_snow = 326.31 #SIPEX2 snow density (kg/m^3)
#d_snow = 305.67 #mean of all EA obs
sd_dsnow = 10 #uncertainty in snow density
d_ice = 915.6 #empirically derived from matching with AUV draft - kg/m^3, unrealistically high!
sd_dice = 10 #uncertainty in ice density
d_water = 1028 #Hutchings2015 - seawater density, kg/m^3
sd_dwater = 1 #uncertainty in seawater density

In [ ]:
#using a loop, recursively iterate fit points toward 'level'

i = 0
while i <= 2:
    #xyzi_, keypoints = fit_points(xyzi_, [10, 2.0], knnf)
    xyzi_, keypoints, redi, coeff = f_points(xyzi_, [20, 2], [1 ,5], 10 )
    print('keypoints: {}'.format(len(keypoints)))
    print('keypoints mean elev: {}'.format(np.mean(keypoints[:,2])))
    print('mean elev: {}'.format(np.mean(xyzi_[:,2])))
    
    print(i)
    i += 1

In [ ]:
##assuming elev > 3m = iceberg!
# or 5m here...
nobergs = np.where(xyzi_[:,2] < 5)
xyzi_ = xyzi_[nobergs[0],:]
input_points = input_points[nobergs[0],:]

In [ ]:
# because water is usually still too high, or too low...
# here we collect the lowest values from the fit point set and use them to
# adjust point heights. *NB this is *after* the fitting-to-surface process,
# and really only required because LiDAR scatters in both directions from
# the intended plane of observation.
adjust = np.percentile(keypoints[:,2],1)
print('adjustment for heights after model fitting: {}'.format(adjust))
xyzi_[:,2] = xyzi_[:,2] + adjust

In [ ]:
#Build a KDtree. Why? so we can query it for point neighbours! The tree is only built
# once, then recycled for as many uses as required. It's a pretty long process
print('building KDtree')
startTime = datetime.now()
points_kdtree = build_tree(xyzi_[:,0:3], 60)
print('time to build tree: {}'.format(datetime.now() - startTime))

In [ ]:
#Query the tree for points within 1 m of each point, and generate mean/median/SD. This
# approximates a median smoothing method (or mean smoothing, or...) in a 2m diameter circle
# placed around each point
startTime = datetime.now()
nhood_stats = np.array(n_filter(xyzi_, points_kdtree, 1))
print('time to generate n stats: {}'.format(datetime.now() - startTime))

#replace xyzi Z with median Z (nhood_stats[:,1])
# or mean_z (nhood_stats[:,0])
xyzi2 = np.column_stack([input_points[:,1], input_points[:,2], \
                            nhood_stats[:,1], input_points[:,4]])

In [ ]:
#Because the tree is already built, collect another dataset smoothed over a bigger radius.
# in this case 5.5 m, to make 11 m radius windows at which resolution correlations between 
# total freeboard and draft begin to make sense (according to Doble et al (2011))
startTime = datetime.now()
r_proxy = np.array(n_filter(xyzi_, points_kdtree, 5.5 ))
print('time to generate n stats: {}'.format(datetime.now() - startTime))

## Part 2: making sea ice thickness!

In [ ]:
#smooth things just a little - assign total freeboard from the 2m diameter nhood filter - in this case median.
tf = nhood_stats[:,1]
#or keep them raw
#tf = xyzi_[:,2]

In [ ]:
#Build traps for points which make no sense.
sub_z = np.where(tf<0)
print('total points where total freeboard < 0 (reassigned to 0): {}'.format(len(sub_z[0])))
tf[sub_z] = 0

#assign the lidar point Z uncertainty to total freeboard
sd_tf = input_points[:, 8]

In [ ]:
#Estimate some snow
zs, zs_uncert = compute_zs(tf, s_i, sd_tf)

#...and filter points which make no sense - negative snow values
sub_z = np.where(zs<0)
print('total points where snow < 0 (reasigned to 0): {}'.format(len(sub_z[0])))
zs[sub_z] = 0

#...and values where 
oversnowed = np.where(zs > tf)
print('total oversnowed points (zs > tf, zs reassigned to 0): {}'.format(len(oversnowed[0])))
zs[oversnowed] = 0


## To do:

- make a pointcloud class
- turn the functions above into methods and functions on the class
- for example (a few methods below):
    
    - pc = pointcloud()
    - pc.medianfilter(args)
    - pc.tolas(args)
    - pc.tonetcdf(args)
    - pc.anglefilter(args)
    - pc.rangegate(args)
    - pc.density(args)
    - pc.grid(args)
    - pc.normals()
    - pc.surf() (maybe)
    - pc.outliers()
    
- and some sea ice specific ones:
    - pc.estimatezs(modelparams)
    - pc.estimatezi(modelparams)
    
- and really later:
    - pc.make(range, angle, trajectory, transforms)
    - pc.computeUncert(args)

All these functions exist, and can be called in various ways - but wrapping them up in a class would be nice and neat.